This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

---

In [1]:
%cd /code

/code


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

---

# Examples

## Find relevant documents to include in prompt

In [ ]:
texts = [
    'This is a document. It has information related to the question I want to ask.',
    'The codeword is `flibberwump`; the answer is `hanzo`.',
    'Here is another document.',
]
question = "What is the answer for the codeword `flibberwump`?"

In [ ]:
from llm_chain.base import Document
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.models import OpenAIEmbeddings

# create a document index (i.e. vector database) and add the text from above.
document_index = ChromaDocumentIndex(
    embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'),
)
document_index.add(docs=[Document(content=x) for x in texts])

In [ ]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIChat
from llm_chain.prompt_templates import DocSearchTemplate

doc_template = DocSearchTemplate(doc_index=document_index, n_docs=1)

# A chain is simply a collection of callables where the output of the previous callable matches
# the input to the next callable.
# Below, the input to the `DocSearchTemplate` is a string (the question) and the output is a
# string (the prompt); and the input to `OpenAIChat` is a string (the prompt).
# Question (str) -> Prompt (str) -> Answer (str)
chain = Chain(links=[
    DocSearchTemplate(doc_index=document_index, n_docs=1),
    OpenAIChat(model_name='gpt-3.5-turbo'),
])
response = chain()
response

In [ ]:
# the chain tracks the usage across any object that has `total_tokens` and `total_cost` properties
print(f"Tokens: {chain.total_tokens:,}")
print(f"Cost: ${chain.total_cost:.6}")

In [ ]:
# you can see the individual costs for the embeddings and the chat
# The embeddings model has 2 records in its history; 
# 1 to embed the original docs and the other to embed the question passed into the chain
for record in chain.history:
    print(record)

In [ ]:
# The chat has model has 1 record in its history
chat_model = chain[1]
print(f"prompt: {chat_model._history[0].prompt}")
print(f"response: {chat_model._history[0].response}")
print(f"cost: {chat_model._history[0].cost}")

---

URL -> doc -> text-splitter -> list[docs] -> vector-db (embeddings) -> None

query -> search -> 

---

In [ ]:
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.models import OpenAIEmbeddings
from llm_chain.chains import Chain
from llm_chain.models import OpenAIChat
from llm_chain.prompt_templates import DocSearchTemplate

# create a document index (i.e. vector database) and add the text from above.
document_index = ChromaDocumentIndex(
    embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'),
)

In [ ]:
def load_text():
    return [
        'This is a document. It has information related to the question I want to ask.',
        'The codeword is `flibberwump`; the answer is `hanzo`.',
        'Here is another document.',
    ]

In [ ]:
# <nothing> -> list[str]
# list[str] -> None
# <ignored> -> str
# str -> str
# str -> str
chain = Chain(links=[
    load_text,
    lambda texts: document_index.add(docs=[Document(content=x) for x in texts]),
    lambda _: "What is the answer for the codeword `flibberwump`?",
    DocSearchTemplate(doc_index=document_index, n_docs=1),
    OpenAIChat(model_name='gpt-3.5-turbo'),
])
response = chain()
response

In [ ]:
# the chain tracks the usage across any object that has `total_tokens` and `total_cost` properties
print(f"Tokens: {chain.total_tokens:,}")
print(f"Cost: ${chain.total_cost:.6}")

In [ ]:
for record in chain.history:
    print(record)

---

In [ ]:
from llm_chain.tools import html_page_loader, split_documents

doc = html_page_loader(url='https://python.langchain.com/en/latest/modules/agents.html')
doc.content

In [ ]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.base import Document
from llm_chain.chains import Chain
from llm_chain.prompt_templates import DocSearchTemplate

chat_model_name = 'gpt-3.5-turbo'
emb_model_name = 'text-embedding-ada-002'

# document_index is used to store and retrieve documents scraped from the URL defined below
# the __call__ function calls add() or search() based on input
document_index = ChromaDocumentIndex(embeddings_model=OpenAIEmbeddings(model_name=emb_model_name))
# prompt_template is retrieves the most relevant docs and stuffs them into the prompt
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=2)
# OpenAI Chat model
chat = OpenAIChat(model_name=chat_model_name)
# converts a string to a list containing a single Document object
text_to_docs = lambda x: [Document(content=x.replace('\n', ' '))]

# questions for ChatGPT; each link in the chain must be a callable
# The first question uses the context from the url via the prior links
ask_question_1 = lambda _: "What is a langchain `Agent`?"
# the second question uses the answer from ChatGPT as part of the prompt
question_2 = lambda x: f'Summarize the following in less than 10 words: "{x}"'

# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    html_page_loader,
    text_to_docs,
    split_documents,  # defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    ask_question_1,
    prompt_template,
    chat,
    question_2,
    chat,
])

response = chain('https://python.langchain.com/en/latest/modules/agents.html')
response

In [ ]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

In [ ]:
for record in chain.history:
    print(record)

---

# Web Search via DuckDuckGo

In [10]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import duckduckgo_search, html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.base import Document
from llm_chain.chains import Chain
from llm_chain.prompt_templates import DocSearchTemplate

# document_index is used to store and retrieve documents scraped from the URL defined below
# the __call__ function calls add() or search() based on the input
document_index = ChromaDocumentIndex(embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'))
# prompt_template is retrieves the most relevant docs and stuffs them into the prompt
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=2)
# OpenAI Chat model
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')
streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo', streaming_callback=lambda x: print(x, end=''))

# for each url, extracts text, cleans, returns doc
def search_results_to_docs(results: list[dict]) -> list[Document]:
    return [Document(content=html_page_loader(x['href']).replace('\n', ' ')) for x in results]

question_1 = "How do I fine-tune an OpenAI Chat model?"

# questions for ChatGPT; each link in the chain must be a callable
# The first question uses the context from the url via the prior links
ask_question_1 = lambda _: question_1
# the second question uses the answer from ChatGPT as part of the prompt
ask_question_2 = lambda x: f'Summarize the following:\n\n{x}"'

# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    duckduckgo_search,
    search_results_to_docs,
    split_documents,  # defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    ask_question_1,
    prompt_template,
    non_streaming_chat,
    ask_question_2,
    streaming_chat,
])
response = chain(question_1)

The provided information suggests that to fine-tune an OpenAI Chat model, one can follow the steps outlined in a blog post that explains the process using Python.

In [11]:
response

'The provided information suggests that to fine-tune an OpenAI Chat model, one can follow the steps outlined in a blog post that explains the process using Python.'

In [12]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost: $0.0027
Tokens: 20,328


In [13]:
for record in chain.history:
    print(record)

timestamp: 2023-06-17 05:00:11.604; cost: $0.001989; total_tokens: 19,892; metadata: {'model_name': 'text-embedding-ada-002'}
timestamp: 2023-06-17 05:00:12.068; cost: $0.000001; total_tokens: 12; metadata: {'model_name': 'text-embedding-ada-002'}
timestamp: 2023-06-17 05:00:14.224; prompt: "Answer the question ..."; response: "To fine-tune an Open...";  cost: $0.000508; total_tokens: 325; metadata: {'model_name': 'gpt-3.5-turbo'}
timestamp: 2023-06-17 05:00:15.871; prompt: "Summarize the follow..."; response: "The provided informa...";  cost: $0.000164; total_tokens: 99; metadata: {'model_name': 'gpt-3.5-turbo'}


In [14]:
from IPython.display import display, Markdown
for index, record in enumerate(chain.message_history):
    display(Markdown(f"## MESSAGE:  {index}"))
    display(Markdown(f"### PROMPT:\n{record.prompt.strip()}"))
    display(Markdown(f"### RESPONSE:\n > {record.response.strip()}"))
    display(Markdown("---"))

## MESSAGE:  0

### PROMPT:
Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
on In this blog post, we've shown you how to fine-tune an OpenAI model using Python. Fine-tuning allows you to adapt the powerful GPT models to your specific tasks, making them more relevant and useful for your applications. By following these steps, you can leverage the capabilities of OpenAI models to create more accurate and task-specific NLP solutions. Happy fine-tuning!   openaichatgptGPT       Saadullah Aleem                            Previous Previous Vectors, Vector Search, and Vector D

enerate human-like text and perform various NLP tasks, such as text generation, translation, summarization, and question-answering. They can even be trained to create vector embeddings to be used in vector search. What is Fine-tuning? Fine-tuning an OpenAI model allows you to adapt the model to your specific task, improving its performance and making it more relevant to your application. In this blog post, we will walk you through the process of fine-tuning an OpenAI model using Python. Prerequi
```

Here is the question:

How do I fine-tune an OpenAI Chat model?

### RESPONSE:
 > To fine-tune an OpenAI Chat model, you can follow the steps outlined in the blog post mentioned in the provided information, which explains the process of fine-tuning an OpenAI model using Python.

---

## MESSAGE:  1

### PROMPT:
Summarize the following information:

To fine-tune an OpenAI Chat model, you can follow the steps outlined in the blog post mentioned in the provided information, which explains the process of fine-tuning an OpenAI model using Python.

### RESPONSE:
 > The provided information suggests that to fine-tune an OpenAI Chat model, one can follow the steps outlined in a blog post that explains the process using Python.

---

---

In [15]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import duckduckgo_search, html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.base import Document
from llm_chain.chains import Chain, Value
from llm_chain.prompt_templates import DocSearchTemplate

document_index = ChromaDocumentIndex(embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'))
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=2)
# OpenAI Chat model
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')

streaming_callback = lambda x: print(x.response, end='')
streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo', streaming_callback=streaming_callback)

# for each url, extracts text, cleans, returns doc
def search_results_to_docs(results: list[dict]) -> list[Document]:
    return [Document(content=html_page_loader(x['href']).replace('\n', ' ')) for x in results]

initial_question = Value()
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'
# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    initial_question,
    duckduckgo_search,
    search_results_to_docs,
    split_documents,  # defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    initial_question,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])

response = chain("What is the meaning of life?")

The meaning of life is complex and subjective, with no definitive answer, and may distract from actually living it.

In [16]:
response

'The meaning of life is complex and subjective, with no definitive answer, and may distract from actually living it.'

In [17]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost: $0.0053
Tokens: 45,536


In [18]:
from IPython.display import display, Markdown
for index, record in enumerate(chain.message_history):
    display(Markdown(f"## MESSAGE:  {index}"))
    display(Markdown(f"### PROMPT:\n{record.prompt.strip()}"))
    display(Markdown(f"### RESPONSE:\n > {record.response.strip()}"))
    display(Markdown("---"))

## MESSAGE:  0

### PROMPT:
Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
the meaning of life at all? To what purpose is it played, this farce in which everything that is essential is irrevocably fixed and determined?[5]Questions about the meaning of life have been expressed in a broad variety of other ways, including: What is the meaning of life? What's it all about? Who are we?[6][7][8] Why are we here? What are we here for?[9][10][11] What is the origin of life?[12] What is the nature of life? What is the nature of reality?[12][13][14] What is the purpose of life? 

to the meaning of life is too profound to be known and understood.[189] You will never live if you are looking for the meaning of life.[161] The meaning of life is to forget about the search for the meaning of life.[161] Ultimately, a person should not ask what the meaning of their life is, but rather must recognize that it is they themselves who are asked. In a word, each person is questioned by life; and they can only answer to life by answering for their own life; to life they can only respon
```

Here is the question:

What is the meaning of life?

### RESPONSE:
 > The meaning of life is a profound question that has been expressed in various ways, but the text suggests that it may be too profound to be fully understood or known. Some perspectives suggest that the search for the meaning of life may be a distraction from actually living it, and that individuals must answer for their own lives in order to find purpose. Therefore, there is no one definitive answer to the question of the meaning of life.

---

## MESSAGE:  1

### PROMPT:
Summarize the following in less than 20 words: "The meaning of life is a profound question that has been expressed in various ways, but the text suggests that it may be too profound to be fully understood or known. Some perspectives suggest that the search for the meaning of life may be a distraction from actually living it, and that individuals must answer for their own lives in order to find purpose. Therefore, there is no one definitive answer to the question of the meaning of life."

### RESPONSE:
 > The meaning of life is complex and subjective, with no definitive answer, and may distract from actually living it.

---

In [12]:
response = chain("What is a langchain document loader?")
response

'A LangChain Document Loader is a versatile tool that loads text from various sources and transforms data for language models.'

In [13]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost: $0.0048
Tokens: 16,495


In [14]:
from IPython.display import display, Markdown
for index, record in enumerate(chain.message_history):
    display(Markdown(f"## MESSAGE:  {index}"))
    display(Markdown(f"### PROMPT:\n{record.prompt.strip()}"))
    display(Markdown(f"### RESPONSE:\n > {record.response.strip()}"))
    display(Markdown("---"))

## MESSAGE:  0

### PROMPT:
Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
f LangChain to build advanced language model applications that are adaptable, efficient, and capable of handling complex use cases.What is a LangChain Agent?A LangChain Agent is an entity that drives decision-making in the framework. It has access to a set of tools and can decide which tool to call based on the user's input. Agents help build complex applications that require adaptive and context-specific responses. They are especially useful when there's an unknown chain of interactions that de

 LangChain enables chains to interact with external data sources to gather data for the generation step. For example, it can help with summarizing long texts or answering questions using specific data sources.Agents: An agent lets an LLM make decisions about actions, take those actions, check the results, and keep going until the job's done. LangChain provides a standard interface for agents, a variety of agents to choose from, and examples of end-to-end agents.Memory: LangChain has a standard i
```

Here is the question:

What is a langchain agent?

### RESPONSE:
 > A LangChain Agent is an entity that drives decision-making in the framework. It has access to a set of tools and can decide which tool to call based on the user's input. Agents help build complex applications that require adaptive and context-specific responses.

---

## MESSAGE:  1

...

---

## MESSAGE:  3

### PROMPT:
Summarize the following in less than 20 words: "A LangChain Document Loader is a versatile tool that can handle various data formats and transform them into a standard structure that language models can easily process. It loads text from various sources and is used in language modeling tasks."

### RESPONSE:
 > A LangChain Document Loader is a versatile tool that loads text from various sources and transforms data for language models.

---

---


# OpenAI Chat

## Simple example showing history and usages/costs

In [ ]:
from llm_chain.models import OpenAIChat

chat = OpenAIChat(model_name='gpt-3.5-turbo', temperature=0)
response = chat("Hi, my name is Shane.")
response

In [ ]:
# the model object tracks usage/cost data across all messages  
def print_usage(model: OpenAIChat):
    usage = f"""
    Total Cost: ${model.total_cost:.6f}
    Total Tokens: {model.total_tokens:,}
    Total Prompt Tokens: {model.total_prompt_tokens:,}
    Total Response Tokens: {model.total_response_tokens:,}
    """
    print(usage)

In [ ]:
print_usage(model=chat)

In [ ]:
# Or you can get the last prompt/response
print(f"previous prompt: {chat.previous_prompt}")
print(f"previous response: {chat.previous_response}")

In [ ]:
# the `history` property contains a list of `MessageMetaData` objects for each message (i.e.
# prompt & response) which contains usage/cost data for that message.
for record in chat.history:
    print(record)

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
response = chat("Do you remember my name?")
response

In [ ]:
for record in chat.history:
    print(record)

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
# You can get the last MessageMetaData via: 
print(f"MessageMetaData: {chat.previous_message}")
# Or you can get the last prompt/response
print(f"previous prompt: {chat.previous_prompt}")
print(f"previous response: {chat.previous_response}")

In [ ]:
print_usage(model=chat)

---

## Memory

The `OpenAIChat` model has a `memory_strategy` parameter and takes a `MemoryBuffer` class. A `MemoryBuffer` class is a callable that takes a `list[MessageMetaData]` (i.e. from the `model.history` property) and also returns a `list[MessageMetaData]` serving as the model's memory (i.e. a list containing the messages that will be sent to the model along with the new prompt). This allows the end user to easily define a memory strategy of their own (e.g. keep the first message and the last `n` messages).

One Example of a `MemoryBuffer` is a `MemoryBufferMessageWindow` class where you can specify the last `n` messages that you want to keep.

In [ ]:
from llm_chain.models import OpenAIChat
from llm_chain.memory import MemoryBufferMessageWindow

chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    temperature=0,
    memory_strategy=MemoryBufferMessageWindow(last_n_messages=0),  # no memory
)
response = chat("Hi, my name is Shane.")
response

In [ ]:
# NOTE: since we created a new OpenAIChat object, the costs/usage are reset
print_usage(model=chat)

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
response = chat("Do you remember my name?")
response

In [ ]:
# we still have access to the full history, but the ChatGPT didn't use any of it.
chat._history

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
# NOTE: since we created a new OpenAIChat object, the costs/usage are reset
print_usage(model=chat)

---

In [4]:
def get_delta(chunk):
    delta = chunk['choices'][0]['delta']
    if 'content' in delta:
        return delta['content']
    return None

In [5]:
import openai
# Example of an OpenAI ChatCompletion request with stream=True
# https://platform.openai.com/docs/guides/chat

# a ChatCompletion request
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': "What is the meaning of life? Answer in one sentence."}
    ],
    temperature=0,
    stream=True,  # this time, we set stream=True
)
print(response)
print(get_delta(next(response)))
message = next(response)
print(get_delta(message))
print(get_delta(next(response)))
print(get_delta(next(response)))

for chunk in response:
    delta = chunk['choices'][0]['delta']
    if 'content' in delta:
        print(delta['content'], end='')
    # print(chunk)


<generator object EngineAPIResource.create.<locals>.<genexpr> at 0xffff8044bcd0>
None
As
 an
 AI
 language model, I do not have personal beliefs or opinions, but the meaning of life is subjective and varies from person to person.

In [6]:
message

<OpenAIObject chat.completion.chunk id=chatcmpl-7SFy2ytujikSVaaIuW4JSOeKWWAT2 at 0xffff8042b950> JSON: {
  "id": "chatcmpl-7SFy2ytujikSVaaIuW4JSOeKWWAT2",
  "object": "chat.completion.chunk",
  "created": 1686968918,
  "model": "gpt-3.5-turbo-0301",
  "choices": [
    {
      "delta": {
        "content": "As"
      },
      "index": 0,
      "finish_reason": null
    }
  ]
}

In [3]:
from llm_chain.models import OpenAIChat

chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    temperature=0,
    streaming_callback=lambda x: print(x.response),
    )
response = chat("Explain what a large language model is in a single sentence.")
# response

A
 large
 language
 model
 is
 a
 type
 of
 artificial
 intelligence
 that
 uses
 deep
 learning
 to
 generate
 human
-like
 language
 and
 understand
 natural
 language
 processing
 tasks
.


In [4]:
# the model object tracks usage/cost data across all messages  
def print_usage(model: OpenAIChat):
    usage = f"""
    Total Cost: ${model.total_cost:.6f}
    Total Tokens: {model.total_tokens:,}
    Total Prompt Tokens: {model.total_prompt_tokens:,}
    Total Response Tokens: {model.total_response_tokens:,}
    """
    print(usage)

print_usage(model=chat)


    Total Cost: $0.000083
    Total Tokens: 47
    Total Prompt Tokens: 21
    Total Response Tokens: 26
    


In [5]:
print(response)

A large language model is a type of artificial intelligence that uses deep learning to generate human-like language and understand natural language processing tasks.


In [6]:
from llm_chain.models import OpenAIChat

chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    temperature=0,
    # streaming_callback=lambda x: print(x.response, end=''),
    )
response = chat("Explain what a large language model is in a single sentence.")
response

'A large language model is a type of artificial intelligence that uses deep learning to generate human-like language and understand natural language processing tasks.'

In [7]:
print_usage(model=chat)


    Total Cost: $0.000100
    Total Tokens: 58
    Total Prompt Tokens: 32
    Total Response Tokens: 26
    


In [8]:
my_string = "Hello"

def update_string():
    global my_string  # Declare the variable as global
    my_string += " World"  # Update the string by appending " World"

# Before calling the function
print(my_string)  # Output: Hello

# Call the function to update the string
update_string()

# After calling the function
print(my_string)  # Output: Hello World


Hello
Hello World


In [26]:
import numpy as np

# Create a random number generator
rng = np.random.default_rng()

# Generate a random list of floats between 0.5 and 13.3
random_floats = rng.uniform(low=-2, high=2, size=50)
random_floats.tolist()

0.9624322322349568